In [467]:
##=============================================================================
## ACMAscrapper V1.0.2
##=============================================================================
##=============================================================================
##
## Scrapper for Register of Radiocommunications Licences of Australian Communications and Media Authority
# 
#@Author Haris Hassan
#@Email haris.hassan@radhaz.com.au 
#@linkedin https://www.linkedin.com/in/hassanharis/
#
##=============================================================================

# Import libraries
import requests
from bs4 import BeautifulSoup
import pandas as pd
pd.set_option('display.max_colwidth', None)

In [ ]:
####Replace Site URL
site_url = 'https://web.acma.gov.au/pls/radcom/site_search.site_lookup?pSITE_ID=9020746'

In [ ]:
ACMApage = requests.get(site_url)
soup = BeautifulSoup(ACMApage.text, 'lxml')
table1 = soup.find('table',{"class": "tablelist responsive"})

In [536]:
ACMAtable = soup.find('table',{"class": "tablelist responsive"})
#links= [linkitem.get('href') for linkitem in table1.find_all('a', attrs={"title": "Assignment Details"})[1:]]
ACMAheaders = [td.text.strip() for td in ACMAtable.select('th')]
ACMAdata = pd.DataFrame(columns = ACMAheaders)

ACMAlinks = []
for j in ACMAtable.find_all('tr')[1:]:
    ACMAValues = [tv.text.strip() for tv in j.find_all('td')]
    ACMAdata.loc[len(ACMAdata)] = ACMAValues
    ACMAlinks.append([tl.get('href') for tl in j.find_all('a')][0])
ACMAdata.insert(1,'links', ACMAlinks )

In [537]:
AntennaTableHeader = ['Licence Number', 'Device Registration ID', 'Client', 'Date Authorised', 'Device Type','Emission Center Frequency', 'Antenna', 'Antenna Height (AGL)', 'Antenna Polarisation','Antenna Azimuth', 'Antenna Tilt']
AntennaTable = pd.DataFrame(columns = AntennaTableHeader)

for acmalink in ACMAdata['links']:
    url = 'https://web.acma.gov.au'+ acmalink
    # Create object page
    page = requests.get(url)
    soup2 = BeautifulSoup(page.text, 'lxml')
    Antennaheaders = [td.text for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(1)')]
    AntennaValues = [td.text.strip() for td in soup2.select_one('table',{"class": "tabledetail"}).select('td:nth-of-type(2)')]
    results = []
    if 'Registration and Application Details - 1' in Antennaheaders:
        Antennaheaders.remove('Registration and Application Details - 1')
    if 'Receiver Details' in Antennaheaders:
        Antennaheaders.remove('Receiver Details')
    if 'Registration and Application Details' in Antennaheaders:
        Antennaheaders.remove('Registration and Application Details')
    if 'Transmitter Details' in Antennaheaders:
        Antennaheaders.remove('Transmitter Details')


    AntennaDictionary = {Antennaheaders[i]: AntennaValues[i] for i in range(len(Antennaheaders))}
    AntennaTable_toAdd = pd.DataFrame([AntennaDictionary])
    AntennaTable = pd.concat([AntennaTable, AntennaTable_toAdd], ignore_index=True)
    #print(output.head())
    
    #AntennaTable = AntennaTable.append(AntennaDictionary, ignore_index=True)


In [538]:
display(AntennaTable[AntennaTableHeader])

,Licence Number,Device Registration ID,Client,Date Authorised,Device Type,Emission Center Frequency,Antenna,Antenna Height (AGL),Antenna Polarisation,Antenna Azimuth,Antenna Tilt
0,10143562,11752611,Optus Mobile Pty Limited,25/Aug/2022,Receiver,1.95 GHz,"Panel(1sector), RRZZT4S4-65B-R6_2100, Commscope",20.77 m,Slant,330°,0°


In [541]:
AntennaTable.to_excel("output.xlsx") 

In [543]:
%notebook "C:\Users\Mewtwo\Documents\Python\ACMAscrapper.ipynb"
